In [90]:
import asyncio
from dotenv import load_dotenv
import os
from enum import Enum
from services.appleSetup import AppleAuth
from services.apple_maps import apple_maps_service
from services.google_maps import google_maps_service
from pydantic import BaseModel
from typing import List, Any, Optional, Dict
import appwrite
from appwrite.client import Client
from appwrite.query import Query
from appwrite.services.users import Users
from appwrite.services.databases import Databases
from appwrite.id import ID
from datetime import datetime
import googlemaps
import logging
from collections import defaultdict
import random
import json
from pprint import pprint
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from fastapi import HTTPException



In [91]:
client = Client()
client.set_endpoint('https://cloud.appwrite.io/v1')
client.set_project('66930c61001b090ab206')
client.set_key(os.getenv('APPWRITE_API_KEY'))
client.set_self_signed()

appwrite_config = {
    "database_id": "66930e1000087eb0d4bd",
    "user_collection_id": "66930e5900107bc194dc",
    "preferences_collection_id": "6696016b00117bbf6352",
    "friends_collection_id": "friends",
    "locations_collection_id": "669d2a590010d4bf7d30",
    "groups_collection_id": "Groups",
    "contact_id": "66a419bb000fa8674a7e"
}

database = Databases(client)
users = Users(client)



In [92]:
class Location(BaseModel):
    lat: float
    lon: float

class ProximityRecommendationRequest(BaseModel):
    locations: List[Location]
    interests: List[str]

class ContentModel(nn.Module):
    def __init__(self, input_dim):
        super(ContentModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc4(x)
        return x

class CollaborativeFilteringModel(nn.Module):
    def __init__(self, num_users, num_items, num_factors):
        super(CollaborativeFilteringModel, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

class HybridModel(nn.Module):
    def __init__(self, content_input_dim, num_users, num_items, num_factors):
        super(HybridModel, self).__init__()
        self.content_model = ContentModel(content_input_dim)
        self.cf_model = CollaborativeFilteringModel(num_users, num_items, num_factors)
        self.fc = nn.Linear(2, 1)  # Combine content and CF scores

    def forward(self, content_input, user, item):
        content_score = self.content_model(content_input)
        cf_score = self.cf_model(user, item)
        combined = torch.cat((content_score, cf_score.unsqueeze(1)), dim=1)
        return self.fc(combined)


In [93]:
def load_model(file_path, num_users, num_items, new_input_dim):
    model_info = torch.load(file_path)
    old_input_dim = model_info['content_input_dim']
    num_factors = model_info['num_factors']

    # Create a new model with the current input dimension
    new_model = HybridModel(
        content_input_dim=new_input_dim,
        num_users=num_users,
        num_items=num_items,
        num_factors=num_factors
    )

    # Load the saved state dict
    old_state_dict = model_info['state_dict']

    # Create a new state dict, copying over compatible parts
    new_state_dict = {}

    # Handle the content model weights
    new_state_dict['content_model.fc1.weight'] = torch.nn.init.xavier_uniform_(
        torch.empty(128, new_input_dim))
    new_state_dict['content_model.fc1.bias'] = old_state_dict['content_model.fc1.bias']
    
    # Copy over the rest of the layers
    for k, v in old_state_dict.items():
        if k not in ['content_model.fc1.weight', 'cf_model.user_factors.weight', 'cf_model.item_factors.weight']:
            new_state_dict[k] = v

    # Initialize new embeddings for users and items
    new_state_dict['cf_model.user_factors.weight'] = torch.nn.init.xavier_uniform_(
        torch.empty(num_users, num_factors))
    new_state_dict['cf_model.item_factors.weight'] = torch.nn.init.xavier_uniform_(
        torch.empty(num_items, num_factors))

    # Load the new state dict
    new_model.load_state_dict(new_state_dict)

    return new_model


In [94]:
class AiModel:
    romantic_date = [
        "Cafe", "Restaurant", "Bakery", "AmusementPark", "Beach", "Winery",
        "Theater", "MovieTheater", "Park", "Zoo", "Aquarium", "Store",
        "MiniGolf", "Bowling", "MusicVenue", "Store", "Mall"
    ]

    family_outing = [
        "AmusementPark", "Zoo", "Aquarium", "Park", "Playground", "MovieTheater",
        "Museum", "NationalPark", "Beach", "Campground", "FoodMarket"
    ]

    outdoor_adventure = [
        "NationalPark", "Park", "Beach", "Hiking", "Kayaking", "Fishing",
        "Golf", "MiniGolf", "RockClimbing", "RVPark", "SkatePark", "Skating",
        "Skiing", "Surfing", "Swimming", "Tennis", "Volleyball"
    ]

    educational_trip = [
        "Museum", "Library", "Aquarium", "NationalPark", "Planetarium", "Zoo",
        "University", "Landmark", "NationalMonument", "ReligiousSite"
    ]

    night_out = [
        "Nightlife", "Brewery", "Restaurant", "MovieTheater", "Theater",
        "MusicVenue", "Casino", "Bar", "Store", "Winery"
    ]

    relaxation_and_wellness = [
        "Beach", "Spa", "FitnessCenter", "Park", "Yoga", "MeditationCenter"
    ]

    sports_and_fitness = [
        "Stadium", "FitnessCenter", "Golf", "Tennis", "Basketball", "Soccer",
        "Baseball", "Swimming", "Volleyball", "Bowling", "RockClimbing",
        "Hiking", "Kayaking", "Surfing", "Skating", "Skiing", "SkatePark"
    ]

    shopping_spree = [
        "Store", "FoodMarket", "Mall", "Pharmacy"
    ]

    kids_fun_day = [
        "AmusementPark", "Zoo", "Aquarium", "Park", "Playground", "MovieTheater",
        "MiniGolf", "Bowling", "Fairground", "GoKart"
    ]

    historical_and_cultural_exploration = [
        "Museum", "Castle", "Fortress", "Landmark", "NationalMonument",
        "ReligiousSite", "Planetarium", "Fairground", "ConventionCenter"
    ]

    vacation = [
        "Hotel", "Beach", "NationalPark", "Park", "Winery", "Campground",
        "Marina", "Skiing", "RVPark", "Store"
    ]

    food_and_drink = [
        "Restaurant", "Cafe", "Bakery", "Brewery", "Winery", "FoodMarket"
    ]

    theme_categories = {
        "romantic_date": romantic_date,
        "family_outing": family_outing,
        "outdoor_adventure": outdoor_adventure,
        "educational_trip": educational_trip,
        "night_out": night_out,
        "relaxation_and_wellness": relaxation_and_wellness,
        "sports_and_fitness": sports_and_fitness,
        "shopping_spree": shopping_spree,
        "kids_fun_day": kids_fun_day,
        "historical_and_cultural_exploration": historical_and_cultural_exploration,
        "vacation": vacation,
        "food_and_drink": food_and_drink
    }

    def __init__(self, users: List[str], location: List[Location], theme: str, other: List[str] = [], budget: int = 100):
        self.users = users
        self.location = location
        self.preferences = None
        self.top_interests = None
        self.locationsList = None
        self.other = other
        self.theme = self.__class__.theme_categories[theme]
        self.places_df = None
        self.user_tensor = None
        self.places_tensor = None
        self.budget = budget
        self.num_users = 0
        self.num_items = 0

    def load_model(self, input_dim):
        self.model = load_model("model.pth", self.num_users, self.num_items, input_dim)
        return self.model

    def get_recommendations(self, user_idx):
        self.model.eval()
        with torch.no_grad():
            current_user_tensor = self.user_tensor[user_idx].unsqueeze(0)
            current_user_tensor_repeated = current_user_tensor.repeat(
                self.places_tensor.shape[0], 1)
            user_place_tensor = torch.cat(
                (current_user_tensor_repeated, self.places_tensor), dim=1)

            user_indices = torch.full(
                (self.places_tensor.shape[0],), user_idx, dtype=torch.long)
            item_indices = torch.arange(
                self.places_tensor.shape[0], dtype=torch.long)

            predictions = self.model(
                user_place_tensor, user_indices, item_indices)
            predictions = predictions.numpy().flatten()

        recommendations = self.places_df.copy()
        recommendations['hybrid_score'] = predictions

        content_scores = self.model.content_model(
            user_place_tensor).detach().numpy().flatten()
        cf_scores = self.model.cf_model(
            user_indices, item_indices).detach().numpy().flatten()

        recommendations['content_score'] = content_scores
        recommendations['cf_score'] = cf_scores

        def normalize_score_hybrid(score):
            return 10 * (score - score.min()) / (score.max() - score.min())

        recommendations['hybrid_score'] = normalize_score_hybrid(
            recommendations['hybrid_score'])

        # Sort recommendations by hybrid score
        recommendations = recommendations.sort_values(
            by='hybrid_score', ascending=False)

        return recommendations

    async def initialize(self):
        # 1. Get preferences
        self.preferences = self.getPreferences(self.users)

        # 2 & 3: Get Top interests
        self.top_interests = self.getTopInterests(self.preferences)

        # 4: Get recommendations
        requestRec = ProximityRecommendationRequest(
            locations=self.location, interests=self.top_interests)
        locs = await self.get_proximity_recommendations(requestRec, self.other)

        # 5: Store recommendations as json
        self.locationsList = (locs['recommendations'])

        # 6: Prepare data for model
        # TODO THIS IS JSUT FOR TESTING
        ratings_data_default = {
            "User": ["User1", "User1", "User2", "User3", "User4", "User4", "User3", "User2"],
            "Address": ["Potomac Pizza", "SeoulSpice", "Mamma Lucia", "National Archives archeological site", "Pebbles Wellness Spa", "Looney's Pub", "University of Maryland Golf Course", "The Cornerstone Grill & Loft"],
            "Rating": [5, 2, 1, 2, 2, 3, 2, 2]
        }

        self.prepare_data(self.locationsList, self.users,
                          self.budget, ratings_data_default)

        # Loading model --- Create mdoel, then train
        input_dim = self.places_tensor.shape[1] + self.user_tensor.shape[1]
        self.model = self.load_model(input_dim)

        # 7: Get recommendations and return it

        #

    @classmethod
    async def create(cls, users: List[str], location: List[Location], theme: str, other: List[str] = [], budget: int = 100):
        instance = cls(users, location, theme, other, budget)
        await instance.initialize()
        return instance

    def getPreferences(self, users: List[str]) -> Dict[str, Any]:
        '''
        # Example usage
        preferences = getPreferences(["66996b7b0025b402922b", "66996d2e00314baa2a20", "669b45980030c00f3b8c", "669c735c001355ea24a7"])
        print(preferences)
        returns:
        defaultdict(<class 'int'>, {'Japanese food': 1, 'Italian food': 2, 'Arcade': 2, 'Parks': 2, 'shopping': 4, 'Culture': 2, 'Golf': 2, 'Indian food': 1, 'British food': 1, 'Eating': 1, 'Beach': 2, 'Museums': 1, 'Football': 1, 'Korean food': 1, 'Spas': 2, 'Go Kart': 2, 'Mexican food': 1, 'Belgian food': 1, 'Pizza food': 1, 'Cinemas': 1, 'Bars': 1, 'Music': 1, 'Theme Parks': 1, 'Nightlife': 1, 'Club': 1, 'Historical Sites': 1, 'Soccer': 1, 'Aquatic Sports': 1, 'Live Sports': 1})


        '''
        res = defaultdict(int)
        for user in users:
            preferences = database.get_document(
                database_id=appwrite_config['database_id'],
                collection_id=appwrite_config['preferences_collection_id'],
                document_id=user
            )
            for key, value in preferences.items():
                if key == 'cuisine':
                    for cuisine in value:
                        res[f"{cuisine} food"] += 1
                elif key == 'entertainment':
                    for entertainment in value:
                        res[f"{entertainment}"] += 1
                elif key == 'shopping':
                    if value == 'YES':
                        res['shopping'] += 1
                elif key == 'learning':
                    for learning in value:
                        res[f"{learning}"] += 1
                elif key == 'sports':
                    for sport in value:
                        res[f"{sport}"] += 1

        return res

    def calculate_centroid(self, locations: List[Location]) -> Dict[str, float]:
        """
        Calculate the centroid of given locations.
        returns: [Location(lat=38.98582939, lon=-76.937329584)]
        """
        latitudes = [loc.lat for loc in locations]
        longitudes = [loc.lon for loc in locations]

        centroid_lat = sum(latitudes) / len(locations)
        centroid_lon = sum(longitudes) / len(locations)

        return {"lat": centroid_lat, "lon": centroid_lon}

    def getTopInterests(self, preferences: Dict[str, int], top_n: int = 10) -> List[str]:
        '''
        Using the preferences from "get preferences" function, get the top N interests
        Example usage:
        interestsPassin = getTopInterests(preferences)
        returns: ['shopping', 'Go Kart', 'Arcade', 'Theme Parks', 'Aquatic Sports', 'Japanese food', 'Museums', 'Football', 'Culture', 'Music']
        '''
        # Filter preferences with values greater than 2
        filtered_preferences = {k: v for k, v in preferences.items() if v > 2}

        sorted_preferences = sorted(
            filtered_preferences.items(), key=lambda item: item[1], reverse=True)

        top_interests = [k for k, v in sorted_preferences[:top_n]]

        # If there are fewer than N keys, randomly select additional keys from the remaining preferences
        if len(top_interests) < top_n:
            remaining_preferences = {
                k: v for k, v in preferences.items() if k not in top_interests}
            additional_interests = random.sample(
                list(remaining_preferences.keys()), top_n - len(top_interests))
            top_interests.extend(additional_interests)

        return top_interests

    async def get_proximity_recommendations(self, request: ProximityRecommendationRequest, other: List[str] = []):
        '''
        Get recommendations based on the user's preferences and location.
        Example usage:
        recommendations = get_proximity_recommendations(ProximityRecommendationRequest(locations=[Location(lat=38.98582939, lon=-76.937329584)], interests=['shopping', 'Go Kart', 'Arcade', 'Theme Parks', 'Aquatic Sports', 'Japanese food', 'Museums', 'Football', 'Culture', 'Music']))
        {'recommendations': [{'name': 'Potomac Pizza', 'address': '7777 Baltimore Ave, College Park, MD  20740, United States', 'location': {'lat': 38.9873178, 'lon': -76.9356036}, 'category': 'Restaurant', 'category2': 'Italian food'}, {'name': 'The Spa at The Hotel at the University of Maryland', 'address': '7777 Baltimore Ave, FL 4, College Park, MD  20740, United States', 

        '''
        centroid = self.calculate_centroid(request.locations)
        try:
            all_recommendations = []
            for interest in request.interests:
                results = await apple_maps_service.search(interest, centroid['lat'], centroid['lon'])
                for result in results:
                    if not result['category']:
                        result['category'] = interest
                    result['category2'] = interest

                all_recommendations.extend(results)
            for otherInterest in other:
                results = await apple_maps_service.search(
                    otherInterest, centroid['lat'], centroid['lon'])
                for result in results:
                    if not result['category']:
                        result['category'] = otherInterest
                    result['category2'] = otherInterest
                all_recommendations.extend(results)

            # Sort recommendations by distance from centroid (if needed)
            sorted_recommendations = sorted(
                all_recommendations,
                key=lambda x: ((x['location']['lat'] - centroid['lat'])**2 +
                               (x['location']['lon'] - centroid['lon'])**2)**0.5
            )

            return {"recommendations": sorted_recommendations}
        except Exception as e:
            raise HTTPException(
                status_code=500, detail=f"Error getting recommendations: {str(e)}")

    def prepare_data(self, data, name, budget, ratingsData=[]):
        '''
        Prepare the data for the model
        Data is the self.locationsList
        '''
        places_df = pd.DataFrame(data)
        self.places_df = places_df
        user_profile = [{
            "Group Name": name,
            "Theme": self.theme,
            "Budget": budget,
            "Other": self.other
        }]
        df_user_profile = pd.DataFrame(user_profile, index=[0])
        places_df['combined_category'] = places_df.apply(
            lambda row: [row['category'], row['category2']], axis=1)
        df_user_profile['combined_preferences'] = df_user_profile.apply(
            lambda row: row['Theme'] + row['Other'], axis=1)
        mlb = MultiLabelBinarizer()
        le_name = LabelEncoder()
        le_address = LabelEncoder()

        # Fit the MultiLabelBinarizer on both places and user preferences
        all_categories = list(places_df['combined_category'].explode().unique(
        )) + list(df_user_profile['combined_preferences'].explode().unique())
        mlb.fit([all_categories])

        # Transform the combined categories
        places_encoded = mlb.transform(places_df['combined_category'])
        user_encoded = mlb.transform(df_user_profile['combined_preferences'])

        # Encode the name and address columns
        places_df['name_encoded'] = le_name.fit_transform(places_df['name'])
        places_df['address_encoded'] = le_address.fit_transform(
            places_df['address'])

        # Extract latitude and longitude
        places_df['lat'] = places_df['location'].apply(lambda x: x['lat'])
        places_df['lon'] = places_df['location'].apply(lambda x: x['lon'])

        # Create the final places feature matrix
        places_features = np.hstack(
            [places_df[['name_encoded', 'address_encoded', 'lat', 'lon']].values, places_encoded])
        user_features = np.hstack(
            [user_encoded, df_user_profile[['Budget']].values])
        self.places_tensor = torch.tensor(places_features, dtype=torch.float32)
        self.user_tensor = torch.tensor(user_features, dtype=torch.float32)

        ratings_df = pd.DataFrame(ratingsData)
        user_encoder = LabelEncoder()
        item_encoder = LabelEncoder()

        ratings_df['user_idx'] = user_encoder.fit_transform(ratings_df['User'])
        ratings_df['item_idx'] = item_encoder.fit_transform(
            ratings_df['Address'])

        # Create the user-item interaction matrix
        self.num_users = len(user_encoder.classes_)
        self.num_items = len(places_df)
        interaction_matrix = np.zeros((self.num_users, self.num_items))

        for _, row in ratings_df.iterrows():
            user_idx = row['user_idx']
            item_idx = row['item_idx']
            rating = row['Rating']
            interaction_matrix[user_idx, item_idx] = rating

        # Convert interaction matrix to tensor
        interaction_tensor = torch.tensor(
            interaction_matrix, dtype=torch.float32)
        return self.places_tensor, self.user_tensor, interaction_tensor



In [95]:
users = ["66996b7b0025b402922b", "66996d2e00314baa2a20"]
locations = [Location(lat=38.98582939, lon=-76.937329584)]

ai = AiModel(users, locations,  "shopping_spree", ["Japanese food"], budget=100)

In [96]:
#Initialize method here
ai.preferences = ai.getPreferences(ai.users)

# 2 & 3: Get Top interests
ai.top_interests = ai.getTopInterests(ai.preferences)

# 4: Get recommendations
requestRec = ProximityRecommendationRequest(
    locations=ai.location, interests=ai.top_interests)
locs = await ai.get_proximity_recommendations(requestRec, ai.other)

# 5: Store recommendations as json
ai.locationsList = (locs['recommendations'])

ai.locationsList


/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/miniconda3/envs/headStarterBackend/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'name': 'Potomac Pizza',
  'address': '7777 Baltimore Ave, College Park, MD  20740, United States',
  'location': {'lat': 38.9873178, 'lon': -76.9356036},
  'category': 'Restaurant',
  'category2': 'Italian food'},
 {'name': 'Iron Rooster - College Park',
  'address': '7777 Baltimore Ave, College Park, MD  20740, United States',
  'location': {'lat': 38.9869586, 'lon': -76.935151},
  'category': 'Restaurant',
  'category2': 'Eating'},
 {'name': 'GrillMarx Steakhouse & Raw Bar',
  'address': '7777 Baltimore Ave, College Park, MD  20740, United States',
  'location': {'lat': 38.9869586, 'lon': -76.935151},
  'category': 'Restaurant',
  'category2': 'Eating'},
 {'name': 'The Hall CP',
  'address': '4656 Hotel Drive, College Park, MD 20742, United States',
  'location': {'lat': 38.9861878, 'lon': -76.9336134},
  'category': 'Restaurant',
  'category2': 'Eating'},
 {'name': 'Wasabi Bistro Japanese Food & Bubble Tea',
  'address': '4505 College Ave, College Park, MD  20740, United States',

In [97]:
ratings_data_default = {
            "User": ["User1", "User1", "User2", "User3", "User4", "User4", "User3", "User2"],
            "Address": ["Potomac Pizza", "SeoulSpice", "Mamma Lucia", "National Archives archeological site", "Pebbles Wellness Spa", "Looney's Pub", "University of Maryland Golf Course", "The Cornerstone Grill & Loft"],
            "Rating": [5, 2, 1, 2, 2, 3, 2, 2]
        }

data = ai.prepare_data(ai.locationsList, ai.users,
                    ai.budget, ratings_data_default)

data

(tensor([[54.0000, 64.0000, 38.9873,  ...,  0.0000,  0.0000,  0.0000],
         [33.0000, 64.0000, 38.9870,  ...,  0.0000,  0.0000,  0.0000],
         [27.0000, 64.0000, 38.9870,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [28.0000, 59.0000, 39.3640,  ...,  0.0000,  0.0000,  0.0000],
         [14.0000, 24.0000, 38.2547,  ...,  0.0000,  0.0000,  0.0000],
         [78.0000,  0.0000, 39.3319,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   1.,
            0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0., 100.]]),
 tensor([[0., 0., 0., 0., 5., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [98]:
input_dim = ai.places_tensor.shape[1] + ai.user_tensor.shape[1]
ai.model = ai.load_model(input_dim)

ai.model
input_dim
ai.model

/var/folders/dv/8r8942l51v104x9yl5sywbtm0000gn/T/ipykernel_69730/135333478.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_info = torch.load(file_path)


HybridModel(
  (content_model): ContentModel(
    (fc1): Linear(in_features=47, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc4): Linear(in_features=64, out_features=1, bias=True)
  )
  (cf_model): CollaborativeFilteringModel(
    (user_factors): Embedding(4, 20)
    (item_factors): Embedding(94, 20)
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [99]:
recs = ai.get_recommendations(0)
recs

,name,address,location,category,category2,combined_category,name_encoded,address_encoded,lat,lon,hybrid_score,content_score,cf_score
93,Woodland Beach,"100 Beach Ave, Smyrna, DE 19977, United States","{'lat': 39.331866, 'lon': -75.4716325}",Beach,Beach,"[Beach, Beach]",78,0,39.331866,-75.471632,10.000000,1.132971,-0.155970
64,Westfield Wheaton,"11160 Veirs Mill Rd, Wheaton, MD 20902, United...","{'lat': 39.0371255, 'lon': -77.0552752}",Store,shopping,"[Store, shopping]",77,7,39.037126,-77.055275,9.690778,1.049356,-0.089432
51,The Queen Vic British Pub,"1206 H St NE, Washington, DC 20002, United St...","{'lat': 38.9004268, 'lon': -76.9898665}",Nightlife,British food,"[Nightlife, British food]",69,11,38.900427,-76.989867,9.155001,0.904588,0.061627
80,Vr Zone Dc,"2300 Wisconsin Ave NW, Unit G - 101, Washingto...","{'lat': 38.9201458, 'lon': -77.0719399}",Arcade,Arcade,"[Arcade, Arcade]",75,23,38.920146,-77.071940,8.804961,0.809687,0.056049
36,Teriyaki express Japanese Grill,"1425 University Blvd E, Hyattsville, MD 20783...","{'lat': 38.9854885, 'lon': -76.9830082}",Restaurant,Japanese food,"[Restaurant, Japanese food]",64,16,38.985489,-76.983008,8.461316,0.716568,0.066225
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,Citizens & Culture,"8113 Georgia Ave, Silver Spring, MD 20910, Un...","{'lat': 38.9909441, 'lon': -77.0262427}",Restaurant,Culture,"[Restaurant, Culture]",10,67,38.990944,-77.026243,0.925128,-1.326691,-0.083541
58,CityCenterDC,"825 Tenth St NW, Washington, DC 20001, United ...","{'lat': 38.9004601, 'lon': -77.0255113}",Store,shopping,"[Store, shopping]",11,69,38.900460,-77.025511,0.897022,-1.334001,0.017408
48,Downtown Silver Spring,"916 Ellsworth Dr, Silver Spring, MD 20910, Un...","{'lat': 38.9965614, 'lon': -77.0245349}",Store,shopping,"[Store, shopping]",18,76,38.996561,-77.024535,0.876956,-1.339530,-0.012229
85,Foothill St,"Foothill St, Woodbridge, VA 22192, United States","{'lat': 38.691924520977786, 'lon': -77.3075897...",Football,Football,"[Football, Football]",23,79,38.691925,-77.307590,0.852713,-1.346357,-0.095650


In [2]:
df = [{'name': 'RVPark', 'address': '9805 Statesville Rd, Ste 6105, Charlotte, NC  28269, United States', 'location': {'lat': 35.3554136, 'lon': -80.8412097}, 'category': 'RVPark', 'category2': 'RVPark', 'combined_category': ['RVPark', 'RVPark'], 'name_encoded': 46, 'address_encoded': 56, 'lat': 35.3554136, 'lon': -80.8412097, 'hybrid_score': 10.0, 'content_score': -1.4295982122421265, 'cf_score': 1.012136459350586}, {'name': 'Snyder Park', 'address': '9401 Snyder Ln, Perry Hall, MD  21128, United States', 'location': {'lat': 39.4045656, 'lon': -76.4571619}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 50, 'address_encoded': 52, 'lat': 39.4045656, 'lon': -76.4571619, 'hybrid_score': 10.0, 'content_score': -1.281294822692871, 'cf_score': 0.5924538373947144}, {'name': 'Gunpowder Falls State Park: Hammerman Area', 'address': '7200 Graces Quarters, Middle River, MD 21220, United States', 'location': {'lat': 39.361498, 'lon': -76.3487289}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 19, 'address_encoded': 26, 'lat': 39.361498, 'lon': -76.3487289, 'hybrid_score': 10.0, 'content_score': -1.0696536302566528, 'cf_score': 1.1378002166748047}, {'name': 'Mariner Point Park', 'address': '100 Kearney Dr, Joppa, MD 21040, United States', 'location': {'lat': 39.3995492, 'lon': -76.3497274}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 30, 'address_encoded': 1, 'lat': 39.3995492, 'lon': -76.3497274, 'hybrid_score': 10.0, 'content_score': -1.238200306892395, 'cf_score': 0.7034976482391357}, {'name': 'Towson Town Center', 'address': '825 Dulaney Valley Rd, Towson, MD 21204, United States', 'location': {'lat': 39.403916, 'lon': -76.599308}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 54, 'address_encoded': 38, 'lat': 39.403916, 'lon': -76.599308, 'hybrid_score': 10.0, 'content_score': -1.253501534461975, 'cf_score': 0.6640702486038208}, {'name': 'Cromwell Valley Park', 'address': '2002 Cromwell Bridge Rd, Parkville, MD  21234, United States', 'location': {'lat': 39.4155754, 'lon': -76.5521573}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 11, 'address_encoded': 8, 'lat': 39.4155754, 'lon': -76.5521573, 'hybrid_score': 10.0, 'content_score': -1.1717935800552368, 'cf_score': 0.8746111989021301}, {'name': 'Putty Hill Park', 'address': '8600 Hoerner Ave, Parkville, MD 21234, United States', 'location': {'lat': 39.3896995, 'lon': -76.5493512}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 44, 'address_encoded': 39, 'lat': 39.3896995, 'lon': -76.5493512, 'hybrid_score': 10.0, 'content_score': -1.267722249031067, 'cf_score': 0.6274270415306091}, {'name': 'North Plaza Shopping Center', 'address': '8898 Waltham Woods Rd, Parkville, MD  21234, United States', 'location': {'lat': 39.4015633, 'lon': -76.5421406}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 36, 'address_encoded': 48, 'lat': 39.4015633, 'lon': -76.5421406, 'hybrid_score': 10.0, 'content_score': -1.2791143655776978, 'cf_score': 0.5980722308158875}, {'name': 'Double Rock Park', 'address': '8211 Glen Rd, Baltimore, MD 21234, United States', 'location': {'lat': 39.3717789, 'lon': -76.526343}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 13, 'address_encoded': 36, 'lat': 39.3717789, 'lon': -76.526343, 'hybrid_score': 10.0, 'content_score': -1.1599884033203125, 'cf_score': 0.9050301313400269}, {'name': 'Angel Park', 'address': '9685 Honeygo Blvd, Perry Hall, MD  21128, United States', 'location': {'lat': 39.4142484, 'lon': -76.4499892}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 2, 'address_encoded': 55, 'lat': 39.4142484, 'lon': -76.4499892, 'hybrid_score': 10.0, 'content_score': -1.3577733039855957, 'cf_score': 0.39538776874542236}, {'name': 'The Avenue at White Marsh', 'address': '8125 Honeygo Blvd, Nottingham, MD 21236, United States', 'location': {'lat': 39.3708104, 'lon': -76.4646077}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 51, 'address_encoded': 31, 'lat': 39.3708104, 'lon': -76.4646077, 'hybrid_score': 10.0, 'content_score': -1.2426320314407349, 'cf_score': 0.6920781135559082}, {'name': 'Nike Clearance Store', 'address': '8115A Honeygo Blvd, Baltimore, MD 21236, United States', 'location': {'lat': 39.3707109, 'lon': -76.467297}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 35, 'address_encoded': 30, 'lat': 39.3707109, 'lon': -76.467297, 'hybrid_score': 10.0, 'content_score': -1.217584252357483, 'cf_score': 0.7566201090812683}, {'name': 'Honeygo Run Regional Park', 'address': '9033 Honeygo Blvd, Perry Hall, MD  21128, United States', 'location': {'lat': 39.3960512, 'lon': -76.4510357}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 20, 'address_encoded': 51, 'lat': 39.3960512, 'lon': -76.4510357, 'hybrid_score': 9.0, 'content_score': -1.2823745012283325, 'cf_score': 0.5896718502044678}, {'name': 'White Marsh Mall', 'address': '8200 Perry Hall Blvd, Nottingham, MD  21236, United States', 'location': {'lat': 39.3749111, 'lon': -76.4668183}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 56, 'address_encoded': 34, 'lat': 39.3749111, 'lon': -76.4668183, 'hybrid_score': 9.0, 'content_score': -1.2454562187194824, 'cf_score': 0.684800922870636}, {'name': 'Belmont Park', 'address': '8701 Walther Blvd, Parkville, MD  21236, United States', 'location': {'lat': 39.3921461, 'lon': -76.5031457}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 5, 'address_encoded': 41, 'lat': 39.3921461, 'lon': -76.5031457, 'hybrid_score': 9.0, 'content_score': -1.1849247217178345, 'cf_score': 0.8407756090164185}, {'name': 'Nottingham Park', 'address': '5146-5154 King Ave, Rosedale, MD 21237, United States', 'location': {'lat': 39.3596936, 'lon': -76.4729948}, 'category': 'Park', 'category2': 'Parks', 'combined_category': ['Park', 'Parks'], 'name_encoded': 37, 'address_encoded': 23, 'lat': 39.3596936, 'lon': -76.4729948, 'hybrid_score': 9.0, 'content_score': -1.2174488306045532, 'cf_score': 0.7569689154624939}, {'name': 'Perry Hall Square Shopping Center', 'address': '4313 Ebenezer Rd, Nottingham, MD  21236, United States', 'location': {'lat': 39.3971223, 'lon': -76.4782912}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 41, 'address_encoded': 19, 'lat': 39.3971223, 'lon': -76.4782912, 'hybrid_score': 9.0, 'content_score': -1.1906594038009644, 'cf_score': 0.8259986639022827}, {'name': 'Tobacco Master', 'address': '4132 E Joppa Rd, Ste 108, Nottingham, MD  21236, United States', 'location': {'lat': 39.399416, 'lon': -76.4825011}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 53, 'address_encoded': 17, 'lat': 39.399416, 'lon': -76.4825011, 'hybrid_score': 9.0, 'content_score': -1.2552189826965332, 'cf_score': 0.6596446633338928}, {'name': 'Goodwill', 'address': '8818 Belair Rd, Nottingham, MD  21236, United States', 'location': {'lat': 39.394448, 'lon': -76.487048}, 'category': 'Store', 'category2': 'shopping', 'combined_category': ['Store', 'shopping'], 'name_encoded': 17, 'address_encoded': 46, 'lat': 39.394448, 'lon': -76.487048, 'hybrid_score': 9.0, 'content_score': -1.184494137763977, 'cf_score': 0.8418849110603333}, {'name': 'Fox Hollow Golf Course', 'address': '1 Cardigan Rd, Timonium, MD 21093, United States', 'location': {'lat': 39.4622403, 'lon': -76.6160005}, 'category': 'Golf', 'category2': 'Golf', 'combined_category': ['Golf', 'Golf'], 'name_encoded': 14, 'address_encoded': 0, 'lat': 39.4622403, 'lon': -76.6160005, 'hybrid_score': 9.0, 'content_score': -1.05552077293396, 'cf_score': -0.7615554332733154}, {'name': "Fratello's Pizza", 'address': '4117 E Joppa Rd, Perry Hall, MD 21236, United States', 'location': {'lat': 39.3976076, 'lon': -76.4839228}, 'category': 'Restaurant', 'category2': 'Italian food', 'combined_category': ['Restaurant', 'Italian food'], 'name_encoded': 15, 'address_encoded': 15, 'lat': 39.3976076, 'lon': -76.4839228, 'hybrid_score': 9.0, 'content_score': -0.8504151105880737, 'cf_score': -0.23304961621761322}, {'name': 'Greystone Golf Course', 'address': '2115 White Hall Rd, White Hall, MD  21161, United States', 'location': {'lat': 39.6339493, 'lon': -76.6078644}, 'category': 'Golf', 'category2': 'Golf', 'combined_category': ['Golf', 'Golf'], 'name_encoded': 18, 'address_encoded': 10, 'lat': 39.6339493, 'lon': -76.6078644, 'hybrid_score': 9.0, 'content_score': -0.9695577621459961, 'cf_score': -0.5400502681732178}, {'name': 'Clifton Park Golf Course', 'address': '2701 St Lo Dr, Baltimore, MD 21213, United States', 'location': {'lat': 39.3229548, 'lon': -76.5824309}, 'category': 'Golf', 'category2': 'Golf', 'combined_category': ['Golf', 'Golf'], 'name_encoded': 10, 'address_encoded': 11, 'lat': 39.3229548, 'lon': -76.5824309, 'hybrid_score': 9.0, 'content_score': -0.9510568380355835, 'cf_score': -0.49237823486328125}, {'name': 'Mount Pleasant Golf Course', 'address': '6001 Hillen Rd, Baltimore, MD 21239, United States', 'location': {'lat': 39.363095, 'lon': -76.5762198}, 'category': 'Golf', 'category2': 'Golf', 'combined_category': ['Golf', 'Golf'], 'name_encoded': 33, 'address_encoded': 25, 'lat': 39.363095, 'lon': -76.5762198, 'hybrid_score': 9.0, 'content_score': -1.0588687658309937, 'cf_score': -0.7701823711395264}, {'name': 'Pine Ridge Golf Course', 'address': '2101 Dulaney Valley Rd, Lutherville, MD 21093, United States', 'location': {'lat': 39.4440762, 'lon': -76.5791251}, 'category': 'Golf', 'category2': 'Golf', 'combined_category': ['Golf', 'Golf'], 'name_encoded': 42, 'address_encoded': 9, 'lat': 39.4440762, 'lon': -76.5791251, 'hybrid_score': 8.0, 'content_score': -1.0290862321853638, 'cf_score': -0.693440318107605}, {'name': 'Ultrazone Baltimore - Eastpoint Mall', 'address': '7835 Eastpoint Mall, Baltimore, MD 21224, United States', 'location': {'lat': 39.2931075, 'lon': -76.5085576}, 'category': 'Arcade', 'category2': 'Arcade', 'combined_category': ['Arcade', 'Arcade'], 'name_encoded': 55, 'address_encoded': 29, 'lat': 39.2931075, 'lon': -76.5085576, 'hybrid_score': 8.0, 'content_score': -1.0640020370483398, 'cf_score': -0.783409595489502}, {'name': 'Bel Air Golf', 'address': '3103 Bel Air Road, Kingsville, MD 21087, United States', 'location': {'lat': 39.4796752, 'lon': -76.4047468}, 'category': 'MiniGolf', 'category2': 'Golf', 'combined_category': ['MiniGolf', 'Golf'], 'name_encoded': 4, 'address_encoded': 12, 'lat': 39.4796752, 'lon': -76.4047468, 'hybrid_score': 8.0, 'content_score': -0.9006653428077698, 'cf_score': -0.36253178119659424}]
df


[{'name': 'RVPark',
  'address': '9805 Statesville Rd, Ste 6105, Charlotte, NC  28269, United States',
  'location': {'lat': 35.3554136, 'lon': -80.8412097},
  'category': 'RVPark',
  'category2': 'RVPark',
  'combined_category': ['RVPark', 'RVPark'],
  'name_encoded': 46,
  'address_encoded': 56,
  'lat': 35.3554136,
  'lon': -80.8412097,
  'hybrid_score': 10.0,
  'content_score': -1.4295982122421265,
  'cf_score': 1.012136459350586},
 {'name': 'Snyder Park',
  'address': '9401 Snyder Ln, Perry Hall, MD  21128, United States',
  'location': {'lat': 39.4045656, 'lon': -76.4571619},
  'category': 'Park',
  'category2': 'Parks',
  'combined_category': ['Park', 'Parks'],
  'name_encoded': 50,
  'address_encoded': 52,
  'lat': 39.4045656,
  'lon': -76.4571619,
  'hybrid_score': 10.0,
  'content_score': -1.281294822692871,
  'cf_score': 0.5924538373947144},
 {'name': 'Gunpowder Falls State Park: Hammerman Area',
  'address': '7200 Graces Quarters, Middle River, MD 21220, United States',
  '